# Modelo


El siguiente código detalla la programación del modelo que aparece en el TFM **"Innovación, precios y dinámica industrial. Un modelo computacional de dos sectores" Autor: Jorge Casinos Directores: Isabel Almudí y Francisco Fatás**

Para comprender las dinámicas del modelo en profundidad se recomienda consultarlo junto con el **psuedocódigo**

### Algoritmos para computa el modelo:

Primero programamos una clase que contendrá los datos del Sector 1 y donde se irán almacenando las empresas y se producirá la interacción entre ellas

In [1]:
import numpy as np
from itertools import compress




class Sector1:
    """
    Clase que contiene todas las empresas del Sector 1
    """

    def __init__(self, alpha_1 = 0.5,     # Sensibilidad calidad/precio de la demanda
                 eta = 1.5,               # Parámetro común en la rutina de fijación de precios
                 c = 0.01,                # Costes de producción unitarios
                 phi = 0.5,               # Condiciones de apropiabilidad
                 epsilon = 0.75,          # Coste de entrada para los imitadores
                 landa = 0.05):           # Probabilidad de entrar realizando innovaciones


        self.alpha, self.eta, self.c, self.phi = alpha_1, eta, c, phi
        self.epsilon, self.landa = epsilon, landa



        self.firms = list()  #lista que contiene todas las empresas del sector

        #listas para guardar las series temporales
        self.m_prices = list()
        self.herfin = list()
        self.m_RD = list()
        self.m_r = list()
        self.num_firms = list()

    def shares(self):                    #Devuelve un vector con todas las cuotas de mercado

        return np.array([firm.s for firm in self.firms])

    def x_max(self):                      # Devuelve el nivel máximo de x

        return max([firm.x for firm in self.firms])

    def x_min(self):                      # Devuelve el nivel mínimo de x

        return min([firm.x for firm in self.firms])

    def R_max(self):                      # Devuelve el gasto máximo en I+D

        return max([firm.R_D for firm in self.firms])

    def mean_knowledge(self):            # Devuelve el conocimiento generado medio

        knowledge = np.array([firm.gamma for firm in self.firms])
        xs = np.array([firm.x for firm in self.firms])

        return np.sum(xs * knowledge)

    def x_max_prev(self):                # Devuelve el nivel máximo de x en t-1

        qualities = list()
        for firm in self.firms:
            if len(firm.xs) != 0:
                qualities.append(firm.xs[-1])

        return max(qualities)

    def medium_prices(self):             # Guarda el precio medio

        prices = np.array([firm.p for firm in self.firms])
        self.m_prices.append(np.sum(self.shares() * prices))

    def medium_RD(self):                 # Guarda el gasto medio en I+D

        RDs = np.array([firm.R_D for firm in self.firms])
        self.m_RD.append(np.sum(self.shares() * RDs))

    def medium_r(self):                  # Guarda el r medio

        rs = np.array([firm.r for firm in self.firms])
        self.m_r.append(np.sum(self.shares() * rs))

    def cal_herfin(self):               # Guarda el índice Herfindhal

        herfin_index = np.sum((self.shares() * 100)**2)
        self.herfin.append(herfin_index)

    def count_firms(self):              # Guarda el número de empresas

        self.num_firms.append(len(self.firms))

Generaremos una clase análoga para el Sector 2

In [2]:

class Sector2:
    """
    Clase que contiene todas las empresas del Sector 2
    """

    def __init__(self, alpha_2=0.5,      # Sensibilidad calidad/precio de la demanda
                 landa=0.05,             # Probabilidad de entrar innovando
                 delta= 1.06,            # Parámetro común en la rutina de precios
                 a=3,                    # Parámetro a de la distribución Beta
                 b=1):                   # Parámetro b de la distribución Beta


        self.alpha, self.landa, self.delta = alpha_2, landa, delta,
        self.a, self.b = a, b

        self.dumb = 0

        self.firms = list()               # Crea una lista para guardar las empresas

        self.m_prices = list()
        self.herfin = list()
        self.num_firms = list()

    def shares(self):                   # Devuelve las cuotas de mercado

        return np.array([firm.s for firm in self.firms])

    def y_max(self):                    # Devuelve la cuota de mercado máxima

        return max([firm.y for firm in self.firms])

    def p_max(self):                    # Devuelve el precio máximo del sector

        return max([firm.p for firm in self.firms])

    def mean_fit(self):                 # Devuelve la competitividad media

        fits = np.array([firm.f for firm in self.firms])
        shares = np.array([firm.s for firm in self.firms])

        return np.sum(fits * shares)

    def medium_prices(self):            # Devuelve y guarda el precio medio

        prices = np.array([firm.p for firm in self.firms])
        self.m_prices.append(np.sum(self.shares() *prices))

    def cal_herfin(self):               # Devuelve y guarda el índice de Herfindhal

        herfin_index = np.sum((self.shares() * 100) ** 2)
        self.herfin.append(herfin_index)

    def count_firms(self):              # Cuenta y guarda el número de empresas en el sector

        self.num_firms.append(len(self.firms))



Ahora programamremos una clase que representa a una empresa particular del Sector 1

In [4]:

class firm_S1:

    """
    Genera una empresa de bienes de capital
    """

    def __init__(self, Sector1,         # Define la industria 
                 pioneer=False,         # If == True es la primera empresa en el mercado
                 innovation=False,      # If == False imita los rasgos de otra empresa
                 share = 0):

        # Listas para guardar las series temporales de ciertas variables
        
        self.prices = list()
        self.quantities = list()
        self.profits = list()
        self.shares = list()
        self.xs = list()
        self.periods = 0

        # Valores iniciales de alguans variables
        
        self.profit = 0
        self.s_e = share
        self.s = 0
        self.q = 0
        self.c = 0

        if pioneer == True or innovation == True:  # Selecciona de manera aleatoria los rasgos

            self.r = np.random.uniform(0, 1)
            self.sigma = np.random.uniform(0, 1)
            self.c_e = Sector1.c
            self.R_D = 0

            if pioneer: # X aleatorio cuando la empresa es la primera en entrar al sector 

                self.x = np.random.uniform(0, 1)

            else:                                # X aleatorio cuando no es la primera en entrar en el sector
                self.x = np.random.uniform(0, Sector1.x_max_prev())


        else:                                    # imita los rasgos de otra empresa establecida

            probabilities = np.array([firm.s for firm in Sector1.firms])
            firm_imitate = np.random.choice(Sector1.firms, p = probabilities)

            self.r, self.sigma, self.x = firm_imitate.r, firm_imitate.sigma, firm_imitate.xs[-1]

            self.q_e = firm_imitate.quantities[-1]

            self.R_D = firm_imitate.R_D

            self.c_e = Sector1.c + Sector1.epsilon * (self.R_D / self.q_e)

    def R_D_investment(self):                   # Calcula el gasto en I + D

        self.R_D = self.r * self.profits[-1]

    def expected_c(self, Sector1):             # Calcula los gastos esperados

        self.q_e = self.quantities[-1]
        self.c_e = Sector1.c + (self.R_D / self.q_e)

    def expected_rivals(self, Sector1, creating = False):   # Crea el conjunto de rivales percibidos

        rivals = list()

        # IGUAL SOBRA

        if creating == True:
            for firm in Sector1.firms:
                if firm == self:
                    continue
                else:
                    if abs(firm.x - self.x) <= self.sigma * Sector1.x_max():
                        rivals.append(firm)
                    else:
                        continue
        else:
            for firm in Sector1.firms[:-1]:

                if firm == self:
                    continue

                else:

                    if abs(firm.xs[-1] - self.x) <= self.sigma * Sector1.x_max_prev():

                        rivals.append(firm)

                    else:
                        continue

        self.rivals = rivals

    def mark_up(self, Sector1):              # Calcula el mark - up de Cournot

        for firm in Sector1.firms:

            if len(firm.shares) != 0:
                firm.s_e = firm.shares[-1]

        rivals_shares = sum(firm.s for firm in self.rivals)
        self.mu = (Sector1.eta + rivals_shares) / ((Sector1.eta + rivals_shares) - self.s_e)

    def price(self):                        # Calcula el precio

        self.p = self.mu * self.c_e

    def knowledge(self, Sector1):           # Genera nuevo conocimiento

        imitation = (Sector1.x_max() - self.x) / self.x

        if self.R_D == 0:
            research = 0
        else:
            research = self.R_D / Sector1.R_max()

        innovation = ((Sector1.phi * imitation) + ((1 - Sector1.phi) * research))
        
        if innovation == 0:
            
                self.gamma = 0

        else:

            slope = 1 / innovation

            new_knowledge = 1 - np.random.power(slope)

            self.gamma = new_knowledge


Programamos una clase que representa a una empresa particular del Sector 2

In [5]:
class firm_S2:
    """
    Generates una empresa del Sector 2
    """

    def __init__(self, Sector1,                # Define el Sector 1
                 Sector2,                      # Define el Sector 2
                 pioneer=False,                # If == True es la primera empresa en entrar en el mercado
                 innovation=False,             # If == False la empresa imita a otras empresas
                 share=0):                     # Cuota de mercado inicial

        # Lista para guardar las series temporales de las variables de interes
        
        self.periods = 0
        self.shares = list()
        self.prices = list()
        self.Xs = list()


        self.s = share

        if pioneer == True or innovation == True:  # Rasgos aleatorios

            self.ro = np.random.beta(Sector2.a, Sector2.b)

            self.X = np.random.uniform(Sector1.x_min(), Sector1.x_max())


        else:  # Imita rasgos

            p_shares = np.array([firm.s for firm in Sector2.firms])

            firm_imitate = np.random.choice(Sector2.firms, p = p_shares)

            self.ro, self.X = firm_imitate.ro, firm_imitate.X

    def understandable(self, Sector1, Sector2):            # Crea conjunto de máquinas "entendibles"

        S1_understand = list()

        for firm in Sector1.firms:

            if abs(self.X - firm.x) <= self.ro * Sector1.x_max():

                S1_understand.append(firm)

            else:
                pass

        self.understand = S1_understand

        if len(S1_understand) == 0:

            Sector2.dumb += 1


    def buy_machine(self, Sector1, Sector2):    # Compra una máquina del sector 1


            xs = np.array([firm.x for firm in self.understand], dtype=float)
            prices = np.array([firm.p for firm in self.understand], dtype=float)
            price_param = prices / np.sum(prices)

            probabilities = Sector1.alpha * xs   + (1 - Sector1.alpha) * (1 - price_param)

            probabilities /= np.sum(probabilities)

            buy = np.random.choice(self.understand, p=probabilities)

            self.c, self.X, self.y = buy.p, buy.x, buy.x

            buy.q += 1

    def price(self, Sector2):                   # Rutina de precios

        self.p = (Sector2.delta / (Sector2.delta - self.s)) * self.c

    def fitness(self, Sector2):                 # Competitividad de la empresa

        self.f = Sector2.alpha * (self.y / Sector2.y_max()) + (1 - Sector2.alpha) * (1 - (self.p / Sector2.p_max()))



Programamos unas funciones que dotarán al modelo de dinámico

In [6]:

def understanding_set(Sector1, Sector2):
    """
    Función para generar la dinámica asociada al proceso de generar el conjunto de bienes de capital
    que las empresas del Sector 2 son capaces de entender
    """

    compresor = list()

    for firm in Sector2.firms:
        firm.understandable(Sector1, Sector2)
        if len(firm.understand) == 0:
            compresor.append(0)
        else:
            compresor.append(1)

    Sector2.firms = list(compress(Sector2.firms, compresor))

    recalculate_shares(Sector2)


def recalculate_xs(Sector1):
    """
    Reescala x (nivel tecnológico Sector 2) para que su suma sea igual a la unidad

    """
    total = np.sum(np.array([firm.x for firm in Sector1.firms]))
    for firm in Sector1.firms:
        firm.x /= total


def recalculate_Xs(Sector2):
    """
    Reescala X (stock de conocimiento empresas Sector 2) para que su suma sea igual a la unidad

    """
    total = np.sum(np.array([firm.X for firm in Sector2.firms]))
    for firm in Sector2.firms:
        firm.X /= total



def recalculate_shares(Sector):
    """
    Recalcula las cuotas de mercado para que su suma sea igual a uno
    """

    total_shares = sum([firm.s for firm in Sector.firms])
    for firm in Sector.firms:
        firm.s /= total_shares



def operate_S1(Sector1):
    """
    Genera la dinámica de las empresas en el Sector 1
    """

    if len(Sector1.firms[:-1]) != 0:

        for firm in Sector1.firms[:-1]:   # For all firms except the new entrant
            firm.R_D_investment()
            firm.expected_c(Sector1)

    for firm in Sector1.firms:

        firm.expected_rivals(Sector1)
        firm.mark_up(Sector1)
        firm.price()
        firm.knowledge(Sector1)


def operate_S2(Sector1, Sector2):
    """
    Genera la dinámica de las empresas en el Sector 2
    """
    understanding_set(Sector1, Sector2)

    for firm in Sector2.firms:
        firm.buy_machine(Sector1, Sector2)
        firm.price(Sector2)

    for firm in Sector2.firms:
        firm.fitness(Sector2)



def replicator_S1(Sector1):
    """
    Dinámica replicador del Sector 1
    """

    for firm in Sector1.firms:
        firm.x = (firm.gamma - Sector1.mean_knowledge()) * firm.x + firm.x


def replicator_S2(Sector2):
    """
    Dinámica replicador del Sector 2
    """

    for firm in Sector2.firms:
        firm.s = (firm.f - Sector2.mean_fit()) * firm.s + firm.s

    total_s = sum([firm.s for firm in Sector2.firms])  #Recalculate shares
    recalculate_shares(Sector2)


def entry_Sector1(Sector1):
    """
    Función que genera la dinámica de entrada en el Sector 1
    """
    if len(Sector1.firms) == 0:

        Sector1.firms.append(firm_S1(Sector1, pioneer= True, share = 1/(len(Sector1.firms) + 1 )))

    else:

        innovation = np.random.binomial(1, Sector1.landa)

        if innovation == True:
            Sector1.firms.append(firm_S1(Sector1, innovation=True, share = 1/(len(Sector1.firms) + 1 )))
        else:
            Sector1.firms.append(firm_S1(Sector1, innovation=False, share = 1/(len(Sector1.firms) + 1)))

    recalculate_xs(Sector1)

def entry_Sector2(Sector1, Sector2):
    """
    Función que genera la dinámica de entrada en el Sector 2
    """

    if len(Sector2.firms) == 0:

        Sector2.firms.append(firm_S2(Sector1, Sector2, pioneer = True, share= 0.005))

    else:

        innovation = np.random.binomial(1, Sector2.landa)


        if innovation == True or len(Sector2.firms) == 0:
            Sector2.firms.append(firm_S2(Sector1, Sector2, innovation=True, share= 0.005))

        if innovation == False:
            Sector2.firms.append(firm_S2(Sector1, Sector2, innovation=False, share= 0.005))

    recalculate_shares(Sector2)
    recalculate_Xs(Sector2)

def exit_S1(Sector1):
    """
    FFunción que genera la dinámica de salida en el Sector 1
    """
    total_q = np.sum([x.q for x in Sector1.firms])     #Calculate market shares
    for firm in Sector1.firms:

        if total_q == 0:
            firm.s = 0
        else:
            firm.s = firm.q / total_q


    for firm in Sector1.firms:
        if firm.q == 0:
            firm.profit = 0
        else:
            firm.c = Sector1.c + (firm.R_D / firm.q)
            firm.profit = firm.q * (firm.p - firm.c)

    selector = list()

    for firm in Sector1.firms:
        if firm.profit <= 0:
            selector.append(0)
        else:
            selector.append(1)

    Sector1.firms = list(compress(Sector1.firms, selector))


    recalculate_shares(Sector1)
    recalculate_xs(Sector1)


def exit_S2(Sector2):
    """
    Función que genera la dinámica de salida en el Sector 2
    """
    selector = list()
    for firm in Sector2.firms:
        if firm.s <= 0.005:
            selector.append(0)
        else:
            selector.append(1)

    Sector2.firms = list(compress(Sector2.firms, selector))
    recalculate_shares(Sector2)
    recalculate_Xs(Sector2)

def save_previous(Sector1, Sector2):
    """
    Guarda los valores de t-1 de X y s
    """

    for firm in Sector1.firms:
        firm.xs.append(firm.x)
    for firm in Sector2.firms:
        firm.shares.append(firm.s)

def save_values(Sector1, Sector2):
    """
    Guarda los valores de interes en el periodo t
    """
    Sector1.medium_prices()
    Sector2.medium_prices()
    Sector1.cal_herfin()
    Sector2.cal_herfin()
    Sector1.medium_RD()
    Sector1.medium_r()
    Sector1.count_firms()
    Sector2.count_firms()
    for firm in Sector1.firms:
        firm.prices.append(firm.p)
        firm.quantities.append(firm.q)
        firm.profits.append(firm.profit)
        firm.shares.append(firm.s)
        firm.periods += 1
        firm.q = 0

    for firm in Sector2.firms:
        firm.prices.append(firm.p)


### Algoritmo de simulación

El siguiente algoritmo sirve para simular el modelo:

In [8]:
def simulate(T, spec = 'normal'):
    """
    T = Número de periodos
    spect = Configuración paramétrica
    """
    
    # Configuraciones paramétricas
    value_landa = {'normal': 0.05, 'mark I': 0.95, 'mark II': 0.005}
    value_phi = {'normal': 0.5, 'mark I': 0.75, 'mark II': 0.05}
    value_epsilon = {'normal': 0.75, 'mark I': 0.75, 'mark II': 0.75}
    value_a = {'normal': 3, 'mark I': 5, 'mark II': 1}
    value_b = {'normal': 2, 'mark I': 1, 'mark II': 3}

    S1 = Sector1(landa=value_landa[spec], phi=value_phi[spec], epsilon= value_epsilon[spec])
    S2 = Sector2(landa=value_landa[spec], a = value_a[spec], b = value_b[spec])

# Dinámica del modelo

    for t in range(T):
        entry_Sector1(S1)
        entry_Sector2(S1, S2)
        operate_S1(S1)

        operate_S2(S1, S2)
        save_previous(S1, S2)
        replicator_S1(S1)
        replicator_S2(S2)
        exit_S1(S1)
        exit_S2(S2)
        save_values(S1, S2)

    return S1, S2


Ejemplo, simulamos el modelo 100 periodos en la configuración base:

In [11]:
Sector1, Sector2 = simulate(100)

Sector1 será un objeto/clase que contenga las empresas y atributos del Sector 1

In [13]:
Sector1

Podemos ver cuantas empresas contiene el sector, que al mismo tiempo serán objetos

In [17]:
Sector1.firms

Solo contiene una empresa. De estos objetos también podemos extraer ciertos atributos como los precios medios del Sector 2 en el último periodo

In [19]:
Sector2.m_prices[-1]

0.012709493336182123

Las empresas también serán objetos, vamos a ver cual es el gasto medio en I+D del monopolista en el Sector 1

In [24]:
monopolio = Sector1.firms[0]
monopolio.r

0.6015989091720829